In [1]:
import requests
import json
import pandas as pd

# Get all strains
API_KEY = 'uEH5NtA'

request_url = f"http://strainapi.evanbusse.com/{API_KEY}/strains/search/all"
response = requests.get(request_url)
data = json.loads(response.text)


In [2]:
# Get all strains
strain_names = pd.DataFrame(list(data.keys()), columns =['name'])

# Get strain characteristics
strain_values = list(data.values())
strain_values = pd.DataFrame.from_dict(strain_values)

# Add strain name to the dataframe
strains_raw = pd.concat([strain_names, strain_values], axis=1)

# Take a look at the data
strains_raw.head()

,name,id,race,flavors,effects
0,Afpak,1,hybrid,"[Earthy, Chemical, Pine]","{'positive': ['Relaxed', 'Hungry', 'Happy', 'S..."
1,African,2,sativa,"[Spicy/Herbal, Pungent, Earthy]","{'positive': ['Euphoric', 'Happy', 'Creative',..."
2,Afternoon Delight,3,hybrid,"[Pepper, Flowery, Pine]","{'positive': ['Relaxed', 'Hungry', 'Euphoric',..."
3,Afwreck,4,hybrid,"[Pine, Earthy, Flowery]","{'positive': ['Relaxed', 'Happy', 'Creative', ..."
4,Agent Orange,5,hybrid,"[Citrus, Orange, Sweet]","{'positive': ['Relaxed', 'Euphoric', 'Happy', ..."


In [3]:
# Get a line for each type of effect
strains = strains_raw.effects.apply(pd.Series) \
    .merge(strains_raw, right_index = True, left_index = True) \
    .drop(["effects"], axis = 1)
strains

,positive,negative,medical,name,id,race,flavors
0,"[Relaxed, Hungry, Happy, Sleepy]",[Dizzy],"[Depression, Insomnia, Pain, Stress, Lack of A...",Afpak,1,hybrid,"[Earthy, Chemical, Pine]"
1,"[Euphoric, Happy, Creative, Energetic, Talkative]",[Dry Mouth],"[Depression, Pain, Stress, Lack of Appetite, N...",African,2,sativa,"[Spicy/Herbal, Pungent, Earthy]"
2,"[Relaxed, Hungry, Euphoric, Uplifted, Tingly]","[Dizzy, Dry Mouth, Paranoid]","[Depression, Insomnia, Pain, Stress, Cramps, H...",Afternoon Delight,3,hybrid,"[Pepper, Flowery, Pine]"
3,"[Relaxed, Happy, Creative, Uplifted, Sleepy]","[Dizzy, Dry Mouth, Paranoid, Dry Eyes]","[Pain, Stress, Headache, Fatigue, Headaches, M...",Afwreck,4,hybrid,"[Pine, Earthy, Flowery]"
4,"[Relaxed, Euphoric, Happy, Energetic, Uplifted]","[Dizzy, Dry Mouth, Paranoid, Dry Eyes]","[Depression, Pain, Stress, Nausea, Headache, H...",Agent Orange,5,hybrid,"[Citrus, Orange, Sweet]"
...,...,...,...,...,...,...,...
1965,"[Relaxed, Happy, Energetic, Uplifted, Focused]","[Dizzy, Dry Mouth, Paranoid, Dry Eyes]","[Depression, Insomnia, Pain, Stress, Fatigue]",Zeus OG,2158,hybrid,"[Earthy, Woody, Pine]"
1966,"[Relaxed, Euphoric, Happy, Uplifted, Sleepy]","[Dizzy, Dry Mouth, Dry Eyes]","[Depression, Insomnia, Pain, Stress, Nausea, H...",Zkittlez,2159,indica,"[Sweet, Berry, Grape]"
1967,"[Relaxed, Euphoric, Happy, Sleepy, Giggly]","[Dizzy, Dry Mouth, Paranoid, Dry Eyes]","[Depression, Insomnia, Pain, Stress, Lack of A...",Zombie OG,2160,indica,"[Earthy, Pungent, Sweet]"
1968,"[Relaxed, Hungry, Happy, Uplifted, Giggly]",[],"[Insomnia, Pain, Cramps, Lack of Appetite, Mus...",Zoom Pie,2161,indica,"[Berry, Earthy, Pungent]"


In [4]:
# Get the description for every strain_id
unique_ids = strains['id'].unique()
descriptions = []
for id in unique_ids:
    request_url = f"http://strainapi.evanbusse.com/{API_KEY}/strains/data/desc/{id}"
    response = requests.get(request_url)
    data = json.loads(response.text)
    desc = list(data.values())
    descriptions.append({'id':id, 'description':desc})
descriptions = pd.DataFrame(descriptions)
descriptions.head()

,id,description
0,1,"[Afpak, named for its direct Afghani and Pakis..."
1,2,[African refers to the indigenous varieties of...
2,3,"[Afternoon Delight, created by Colorado Seed I..."
3,4,[Afwreck is a hybrid cross of Afghani and Trai...
4,5,[Don’t let the name scare you! The only herbic...


In [5]:
# Add description to the df
strains = strains.merge(descriptions, on='id')
strains

,positive,negative,medical,name,id,race,flavors,description
0,"[Relaxed, Hungry, Happy, Sleepy]",[Dizzy],"[Depression, Insomnia, Pain, Stress, Lack of A...",Afpak,1,hybrid,"[Earthy, Chemical, Pine]","[Afpak, named for its direct Afghani and Pakis..."
1,"[Euphoric, Happy, Creative, Energetic, Talkative]",[Dry Mouth],"[Depression, Pain, Stress, Lack of Appetite, N...",African,2,sativa,"[Spicy/Herbal, Pungent, Earthy]",[African refers to the indigenous varieties of...
2,"[Relaxed, Hungry, Euphoric, Uplifted, Tingly]","[Dizzy, Dry Mouth, Paranoid]","[Depression, Insomnia, Pain, Stress, Cramps, H...",Afternoon Delight,3,hybrid,"[Pepper, Flowery, Pine]","[Afternoon Delight, created by Colorado Seed I..."
3,"[Relaxed, Happy, Creative, Uplifted, Sleepy]","[Dizzy, Dry Mouth, Paranoid, Dry Eyes]","[Pain, Stress, Headache, Fatigue, Headaches, M...",Afwreck,4,hybrid,"[Pine, Earthy, Flowery]",[Afwreck is a hybrid cross of Afghani and Trai...
4,"[Relaxed, Euphoric, Happy, Energetic, Uplifted]","[Dizzy, Dry Mouth, Paranoid, Dry Eyes]","[Depression, Pain, Stress, Nausea, Headache, H...",Agent Orange,5,hybrid,"[Citrus, Orange, Sweet]",[Don’t let the name scare you! The only herbic...
...,...,...,...,...,...,...,...,...
1965,"[Relaxed, Happy, Energetic, Uplifted, Focused]","[Dizzy, Dry Mouth, Paranoid, Dry Eyes]","[Depression, Insomnia, Pain, Stress, Fatigue]",Zeus OG,2158,hybrid,"[Earthy, Woody, Pine]",[Zeus OG is a hybrid cross between Pineapple O...
1966,"[Relaxed, Euphoric, Happy, Uplifted, Sleepy]","[Dizzy, Dry Mouth, Dry Eyes]","[Depression, Insomnia, Pain, Stress, Nausea, H...",Zkittlez,2159,indica,"[Sweet, Berry, Grape]",[Zkittlez is an indica-dominant mix of Grape A...
1967,"[Relaxed, Euphoric, Happy, Sleepy, Giggly]","[Dizzy, Dry Mouth, Paranoid, Dry Eyes]","[Depression, Insomnia, Pain, Stress, Lack of A...",Zombie OG,2160,indica,"[Earthy, Pungent, Sweet]",[If you’re looking to transform into a flesh-e...
1968,"[Relaxed, Hungry, Happy, Uplifted, Giggly]",[],"[Insomnia, Pain, Cramps, Lack of Appetite, Mus...",Zoom Pie,2161,indica,"[Berry, Earthy, Pungent]",[Zoom Pie (also known as Zombie Pie) is a heav...


In [6]:
# Change column order
cols = ['id', 'name', 'race', 'flavors', 'positive', 'negative', 'medical', 'description']
strains = strains[cols]
strains.head(1)

,id,name,race,flavors,positive,negative,medical,description
0,1,Afpak,hybrid,"[Earthy, Chemical, Pine]","[Relaxed, Hungry, Happy, Sleepy]",[Dizzy],"[Depression, Insomnia, Pain, Stress, Lack of A...","[Afpak, named for its direct Afghani and Pakis..."


In [7]:
# Combine everything in one column
strains['Combined'] = strains[strains.columns[3:]].apply(
    lambda x: ','.join(x.dropna().astype(str)),
    axis=1
)
strains

C:\Users\doina\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,id,name,race,flavors,positive,negative,medical,description,Combined
0,1,Afpak,hybrid,"[Earthy, Chemical, Pine]","[Relaxed, Hungry, Happy, Sleepy]",[Dizzy],"[Depression, Insomnia, Pain, Stress, Lack of A...","[Afpak, named for its direct Afghani and Pakis...","['Earthy', 'Chemical', 'Pine'],['Relaxed', 'Hu..."
1,2,African,sativa,"[Spicy/Herbal, Pungent, Earthy]","[Euphoric, Happy, Creative, Energetic, Talkative]",[Dry Mouth],"[Depression, Pain, Stress, Lack of Appetite, N...",[African refers to the indigenous varieties of...,"['Spicy/Herbal', 'Pungent', 'Earthy'],['Euphor..."
2,3,Afternoon Delight,hybrid,"[Pepper, Flowery, Pine]","[Relaxed, Hungry, Euphoric, Uplifted, Tingly]","[Dizzy, Dry Mouth, Paranoid]","[Depression, Insomnia, Pain, Stress, Cramps, H...","[Afternoon Delight, created by Colorado Seed I...","['Pepper', 'Flowery', 'Pine'],['Relaxed', 'Hun..."
3,4,Afwreck,hybrid,"[Pine, Earthy, Flowery]","[Relaxed, Happy, Creative, Uplifted, Sleepy]","[Dizzy, Dry Mouth, Paranoid, Dry Eyes]","[Pain, Stress, Headache, Fatigue, Headaches, M...",[Afwreck is a hybrid cross of Afghani and Trai...,"['Pine', 'Earthy', 'Flowery'],['Relaxed', 'Hap..."
4,5,Agent Orange,hybrid,"[Citrus, Orange, Sweet]","[Relaxed, Euphoric, Happy, Energetic, Uplifted]","[Dizzy, Dry Mouth, Paranoid, Dry Eyes]","[Depression, Pain, Stress, Nausea, Headache, H...",[Don’t let the name scare you! The only herbic...,"['Citrus', 'Orange', 'Sweet'],['Relaxed', 'Eup..."
...,...,...,...,...,...,...,...,...,...
1965,2158,Zeus OG,hybrid,"[Earthy, Woody, Pine]","[Relaxed, Happy, Energetic, Uplifted, Focused]","[Dizzy, Dry Mouth, Paranoid, Dry Eyes]","[Depression, Insomnia, Pain, Stress, Fatigue]",[Zeus OG is a hybrid cross between Pineapple O...,"['Earthy', 'Woody', 'Pine'],['Relaxed', 'Happy..."
1966,2159,Zkittlez,indica,"[Sweet, Berry, Grape]","[Relaxed, Euphoric, Happy, Uplifted, Sleepy]","[Dizzy, Dry Mouth, Dry Eyes]","[Depression, Insomnia, Pain, Stress, Nausea, H...",[Zkittlez is an indica-dominant mix of Grape A...,"['Sweet', 'Berry', 'Grape'],['Relaxed', 'Eupho..."
1967,2160,Zombie OG,indica,"[Earthy, Pungent, Sweet]","[Relaxed, Euphoric, Happy, Sleepy, Giggly]","[Dizzy, Dry Mouth, Paranoid, Dry Eyes]","[Depression, Insomnia, Pain, Stress, Lack of A...",[If you’re looking to transform into a flesh-e...,"['Earthy', 'Pungent', 'Sweet'],['Relaxed', 'Eu..."
1968,2161,Zoom Pie,indica,"[Berry, Earthy, Pungent]","[Relaxed, Hungry, Happy, Uplifted, Giggly]",[],"[Insomnia, Pain, Cramps, Lack of Appetite, Mus...",[Zoom Pie (also known as Zombie Pie) is a heav...,"['Berry', 'Earthy', 'Pungent'],['Relaxed', 'Hu..."
